In [1]:
from choice_model import ChoiceModel
import os

folder = 'remove_long_term_project'
base_url = 'http://10.31.183.83:11434'
# base_url = 'http://localhost:11434'
remove_type = 'Long-term Project'
# remove_type = None
if not os.path.exists(folder):
    os.makedirs(folder)

old_model = ChoiceModel(base_url=base_url)
old_model.roll_back(0)
init_graph = old_model.graph.copy()
nodes = [n for n in init_graph.nodes if init_graph.nodes[n]['type'] != remove_type]
test_graph = init_graph.subgraph(nodes).copy()

old_model.save_graph(file_name=f"{folder}/{folder}_0.pkl")
old_model.visualize(file_name=f"{folder}/{folder}_0.html")

Saving graph to remove_long_term_project/remove_long_term_project_0.pkl


In [2]:
from choice_model import ChoiceModel


def predict(period,remove_type=None, k1=5,k2=1,top_k=10):
    model = ChoiceModel(base_url=base_url)
    prev_period_file = f'{folder}/{folder}_{period-1}.pkl'
    new_model = ChoiceModel(graph_path=prev_period_file,base_url=base_url) 
    graph = model.get_period(period)
    new_nodes = [n for n in graph.nodes if n not in list(new_model.graph.nodes)]
    new_nodes = [n for n in new_nodes if graph.nodes[n]['type'] != remove_type]
    new_nodes = sorted(new_nodes,key=lambda x: graph.nodes[x]['type'])
    test_graph = new_model.graph.copy()
    print("begin simulation",len(test_graph.nodes),len(test_graph.edges))
    test_graph,failed_nodes = model.predict_links(test_graph, new_nodes, file_name=f"{folder}/{folder}_{period}.pkl",
                                     choice_type=None,k1=k1,k2=k2,top_k=top_k)
    
    if len(failed_nodes) > 0:
        print("retrying failed nodes")
        test_graph,_ = model.predict_links(test_graph, failed_nodes,file_name=f"{folder}/{folder}_{period}.pkl",k1=k1,k2=k2,top_k=top_k)

    issolated_nodes = model.get_isolated_nodes(graph=test_graph)
    if len(issolated_nodes) > 0:
        print("retrying issolated nodes")
        test_graph,_ = model.predict_links(test_graph, issolated_nodes,file_name=f"{folder}/{folder}_{period}.pkl",k1=k1,k2=k2,top_k=top_k)
        test_graph = model.remove_isolated_nodes(graph=test_graph)
        model.save_graph(file_name=f"{folder}/{folder}_{period}.pkl",graph=test_graph)

    new_model = ChoiceModel(graph_path=f"{folder}/{folder}_{period}.pkl")
    new_model.visualize(file_name=f"{folder}/{folder}_{period}.html")
    print("end simulation",len(test_graph.nodes),len(test_graph.edges))
    PPI, CVI, DCI = new_model.evaluate(test_graph)
    print(f"Period {period} PPI: {PPI} CVI: {CVI} DCI: {DCI}")


In [ ]:
for i in range(1,10):
    predict(i,remove_type=remove_type)

begin simulation 99 178


  1%|██                                                                                                                                                       | 1/74 [00:11<13:50, 11.38s/it]

'answer'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [12:05<00:00,  9.80s/it]


Saving graph to remove_long_term_project/remove_long_term_project_1.pkl
retrying failed nodes


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:26<00:00,  8.86s/it]


Saving graph to remove_long_term_project/remove_long_term_project_2.pkl
retrying issolated nodes


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.66s/it]


Saving graph to remove_long_term_project/remove_long_term_project_2.pkl
Removed 0 isolated nodes
Saving graph to remove_long_term_project/remove_long_term_project_2.pkl
end simulation 290 614
Period 2 PPI: 0.457526010809421 CVI: 3.9761904761904763 DCI: 0.08108108108108109
begin simulation 290 614


  4%|█████▋                                                                                                                                                  | 6/162 [01:02<28:41, 11.04s/it]